# **Chapter 15: DevSecOps & Automation**

## Introduction: The Velocity-Security Convergence

In Chapter 14, we secured artificial intelligence systems—implementing differential privacy, federated learning, and agentic guardrails. Yet these controls remain theoretical until embedded within the software delivery pipeline. Manual security reviews cannot match the velocity of modern development, where AI models deploy multiple times daily and infrastructure scales elastically in response to traffic.

**DevSecOps** represents the philosophical and technical integration of security practices within the DevOps lifecycle. It is not merely "DevOps plus security tools" but a cultural transformation where security becomes a shared responsibility across development, operations, and security teams. The goal is **"shifting security left"**—detecting vulnerabilities at the design and coding phases rather than in production—while simultaneously **"shifting right"** through runtime protection and continuous monitoring.

This chapter operationalizes the security controls from previous chapters. You will learn to automate the scanning of container images for CVEs, enforce security policies through code, implement **SLSA (Supply Chain Levels for Software Artifacts)** compliance for artifact provenance, and orchestrate incident response through **SOAR (Security Orchestration, Automation, and Response)** platforms.

By the end, you will architect a CI/CD pipeline where every commit triggers security validation, every artifact is cryptographically signed, and every deployment includes runtime protection—achieving both velocity and resilience.

---

## 15.1 Integrating Security into CI/CD Pipelines

Continuous Integration/Continuous Deployment (CI/CD) pipelines are the arteries of modern software delivery. Integrating security into these pipelines requires balancing speed with rigor—blocking critical vulnerabilities while avoiding "alert fatigue" that leads developers to bypass controls.

### Pipeline Architecture and Security Gates

A secure CI/CD pipeline implements multiple verification stages, failing fast when possible to minimize compute waste.

**Secure Pipeline Stages:**

1. **Pre-commit**: Git hooks, secrets scanning, linting
2. **Build**: SAST (Static Application Security Testing), dependency scanning, license compliance
3. **Test**: Unit tests with security assertions, IAST (Interactive Application Security Testing)
4. **Package**: Container image scanning, artifact signing, SBOM generation
5. **Staging**: DAST (Dynamic Application Security Testing), infrastructure scanning
6. **Deploy**: Policy validation, runtime configuration checks
7. **Production**: Continuous monitoring, drift detection

### GitHub Actions Implementation

GitHub Actions provides native CI/CD with extensive security tooling integration.

**Comprehensive Security Workflow:**
```yaml
# .github/workflows/secure-pipeline.yml
name: Secure CI/CD Pipeline

on:
  push:
    branches: [main, develop]
  pull_request:
    branches: [main]

env:
  REGISTRY: ghcr.io
  IMAGE_NAME: ${{ github.repository }}
  # Fail pipeline on critical vulnerabilities
  SEVERITY_THRESHOLD: critical

jobs:
  # Stage 1: Secrets and Credential Scanning
  secrets-scan:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4
        with:
          fetch-depth: 0  # Full history for secret scanning
      
      - name: Detect Secrets with Gitleaks
        uses: gitleaks/gitleaks-action@v2
        env:
          GITHUB_TOKEN: ${{ secrets.GITHUB_TOKEN }}
          GITLEAKS_LICENSE: ${{ secrets.GITLEAKS_LICENSE }}
      
      - name: Scan for Hardcoded Credentials
        uses: trufflesecurity/trufflehog@main
        with:
          path: ./
          base: main
          head: HEAD
          extra_args: --debug --only-verified

  # Stage 2: Static Analysis (SAST)
  sast-analysis:
    runs-on: ubuntu-latest
    needs: secrets-scan
    strategy:
      matrix:
        language: ['javascript', 'python', 'go']
    
    steps:
      - uses: actions/checkout@v4
      
      - name: Initialize CodeQL
        uses: github/codeql-action/init@v3
        with:
          languages: ${{ matrix.language }}
          queries: security-extended,security-and-quality
      
      - name: Autobuild
        uses: github/codeql-action/autobuild@v3
      
      - name: Perform CodeQL Analysis
        uses: github/codeql-action/analyze@v3
        with:
          category: "/language:${{matrix.language}}"
      
      # Additional SonarCloud for code quality and security hotspots
      - name: SonarCloud Scan
        uses: SonarSource/sonarcloud-github-action@master
        env:
          GITHUB_TOKEN: ${{ secrets.GITHUB_TOKEN }}
          SONAR_TOKEN: ${{ secrets.SONAR_TOKEN }}

  # Stage 3: Dependency and Supply Chain Security
  dependency-check:
    runs-on: ubuntu-latest
    needs: secrets-scan
    steps:
      - uses: actions/checkout@v4
      
      - name: Setup Node.js
        uses: actions/setup-node@v4
        with:
          node-version: '20'
          cache: 'npm'
      
      - name: Install dependencies
        run: npm ci
      
      # NPM Audit with custom config
      - name: NPM Audit
        run: |
          npm audit --audit-level=moderate --production
          if [ $? -ne 0 ]; then
            echo "::error::Vulnerabilities found in dependencies"
            exit 1
          fi
      
      # Snyk for comprehensive SCA
      - name: Snyk Security Scan
        uses: snyk/actions/node@master
        continue-on-error: true  # Don't block, just report
        env:
          SNYK_TOKEN: ${{ secrets.SNYK_TOKEN }}
        with:
          args: --severity-threshold=high --sarif-file-output=snyk.sarif
      
      - name: Upload Snyk results to GitHub Security Tab
        uses: github/codeql-action/upload-sarif@v2
        if: always()
        with:
          sarif_file: snyk.sarif
      
      # Generate and store SBOM
      - name: Generate SBOM
        uses: anchore/sbom-action@v0
        with:
          format: spdx-json
          output-file: sbom.spdx.json
      
      - name: Upload SBOM
        uses: actions/upload-artifact@v3
        with:
          name: sbom
          path: sbom.spdx.json

  # Stage 4: Build and Container Security
  build-and-scan:
    runs-on: ubuntu-latest
    needs: [sast-analysis, dependency-check]
    permissions:
      contents: read
      packages: write
      id-token: write  # For Cosign signing
    
    steps:
      - uses: actions/checkout@v4
      
      - name: Set up Docker Buildx
        uses: docker/setup-buildx-action@v3
      
      - name: Log in to Container Registry
        uses: docker/login-action@v3
        with:
          registry: ${{ env.REGISTRY }}
          username: ${{ github.actor }}
          password: ${{ secrets.GITHUB_TOKEN }}
      
      # Build image with security scanning
      - name: Build and export to Docker
        uses: docker/build-push-action@v5
        with:
          context: .
          load: true
          tags: ${{ env.REGISTRY }}/${{ env.IMAGE_NAME }}:${{ github.sha }}
      
      # Scan with Trivy before pushing
      - name: Scan image with Trivy
        uses: aquasecurity/trivy-action@master
        with:
          image-ref: ${{ env.REGISTRY }}/${{ env.IMAGE_NAME }}:${{ github.sha }}
          format: 'sarif'
          output: 'trivy-results.sarif'
          severity: ${{ env.SEVERITY_THRESHOLD }}
          exit-code: '1'  # Fail build on critical CVEs
      
      - name: Upload Trivy scan results
        uses: github/codeql-action/upload-sarif@v2
        if: always()
        with:
          sarif_file: 'trivy-results.sarif'
      
      # Push only if scan passes
      - name: Push image
        uses: docker/build-push-action@v5
        with:
          context: .
          push: true
          tags: |
            ${{ env.REGISTRY }}/${{ env.IMAGE_NAME }}:${{ github.sha }}
            ${{ env.REGISTRY }}/${{ env.IMAGE_NAME }}:latest
      
      # Sign image with Cosign (keyless)
      - name: Install Cosign
        uses: sigstore/cosign-installer@v3
      
      - name: Sign image
        run: |
          cosign sign --yes \
            ${{ env.REGISTRY }}/${{ env.IMAGE_NAME }}:${{ github.sha }}

  # Stage 5: Infrastructure as Code Scanning
  iac-security:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4
      
      - name: Run Checkov
        uses: bridgecrewio/checkov-action@master
        with:
          directory: .
          framework: terraform,dockerfile,kubernetes
          output_format: sarif
          output_file_path: reports/checkov.sarif
      
      - name: Upload Checkov results
        uses: github/codeql-action/upload-sarif@v2
        if: always()
        with:
          sarif_file: reports/checkov.sarif
      
      # Terraform specific scanning
      - name: Terraform Security Scan
        uses: aquasecurity/tfsec-action@v1.0.0
        with:
          soft_fail: true

  # Stage 6: Dynamic Testing (DAST) - Post-deployment to staging
  dast:
    runs-on: ubuntu-latest
    needs: deploy-staging
    if: github.ref == 'refs/heads/main'
    steps:
      - name: ZAP Full Scan
        uses: zaproxy/action-full-scan@v0.9.0
        with:
          target: 'https://staging.example.com'
          rules_file_name: '.zap/rules.tsv'
          cmd_options: '-a'
      
      - name: Upload ZAP results
        uses: actions/upload-artifact@v3
        with:
          name: zap-report
          path: report_*.html
```

### GitLab CI Implementation

For organizations using GitLab, the pipeline structure follows similar principles with different syntax:

```yaml
# .gitlab-ci.yml
stages:
  - validate
  - build
  - test
  - security
  - deploy

variables:
  DOCKER_DRIVER: overlay2
  DOCKER_TLS_CERTDIR: "/certs"
  SAST_ANALYZER_IMAGE_TAG: "latest"
  SAST_EXCLUDED_PATHS: "spec, test, tests, tmp"

# Template for security scanning
.secrets_scan:
  stage: validate
  image: alpine/git
  script:
    - apk add --no-cache gitleaks
    - gitleaks detect --source . --verbose --redact
  rules:
    - if: $CI_PIPELINE_SOURCE == "merge_request_event"
    - if: $CI_COMMIT_BRANCH == "main"

# SAST using GitLab's built-in analyzer
sast:
  stage: security
  image: returntocorp/semgrep
  script:
    - semgrep --config=auto --json --output=semgrep-report.json .
  artifacts:
    reports:
      sast: semgrep-report.json
    paths:
      - semgrep-report.json
  rules:
    - if: $CI_COMMIT_BRANCH

# Container Scanning with Trivy
container_scanning:
  stage: security
  image: docker:stable
  services:
    - docker:dind
  variables:
    IMAGE: "$CI_REGISTRY_IMAGE:$CI_COMMIT_SHA"
  script:
    - docker pull $IMAGE
    - docker run --rm -v /var/run/docker.sock:/var/run/docker.sock 
      aquasec/trivy image --exit-code 1 --severity HIGH,CRITICAL $IMAGE
  allow_failure: false
  rules:
    - if: $CI_COMMIT_BRANCH == "main"

# Dependency Checking
dependency_scanning:
  stage: security
  image: node:18-alpine
  script:
    - npm audit --audit-level moderate
    - npx audit-ci --moderate
  artifacts:
    reports:
      dependency_scanning: dependency-scan-report.json

# DAST using OWASP ZAP
dast:
  stage: security
  image: owasp/zap2docker-stable
  script:
    - mkdir /zap/wrk/
    - zap-baseline.py -t https://staging.example.com -g gen.conf -r testreport.html
  artifacts:
    when: always
    paths:
      - /zap/wrk/testreport.html
  rules:
    - if: $CI_COMMIT_BRANCH == "main"
```

### Jenkins Pipeline (Enterprise Environments)

For organizations maintaining Jenkins infrastructure:

```groovy
// Jenkinsfile
pipeline {
    agent any
    
    environment {
        DOCKER_REGISTRY = 'registry.company.com'
        IMAGE_NAME = 'secure-app'
        SNYK_TOKEN = credentials('snyk-api-token')
    }
    
    stages {
        stage('Checkout') {
            steps {
                checkout scm
                sh 'git clean -fdx'
            }
        }
        
        stage('Pre-commit Checks') {
            parallel {
                stage('Secrets Scan') {
                    steps {
                        sh '''
                            docker run --rm -v $(pwd):/code \
                            zricethezav/gitleaks:latest detect \
                            --source /code --verbose --redact
                        '''
                    }
                }
                stage('Linting') {
                    steps {
                        sh 'npm run lint'
                        sh 'npm run security-lint'  // ESLint security plugin
                    }
                }
            }
        }
        
        stage('SAST') {
            steps {
                script {
                    // SonarQube analysis
                    withSonarQubeEnv('SonarQube') {
                        sh 'npm run sonar'
                    }
                    
                    // Wait for SonarQube quality gate
                    timeout(time: 10, unit: 'MINUTES') {
                        waitForQualityGate abortPipeline: true
                    }
                }
            }
        }
        
        stage('Dependency Check') {
            steps {
                sh 'npm audit --production --audit-level=moderate'
                
                // OWASP Dependency-Check
                dependencyCheck additionalArguments: '''
                    -o ./dependency-check-report
                    -s ./package.json
                    -f XML
                    --enableExperimental''', 
                    odcInstallation: 'OWASP-Dependency-Check'
                
                dependencyCheckPublisher pattern: 'dependency-check-report/dependency-check-report.xml'
            }
        }
        
        stage('Build') {
            steps {
                script {
                    def image = docker.build("${DOCKER_REGISTRY}/${IMAGE_NAME}:${BUILD_NUMBER}")
                    
                    // Scan before pushing
                    sh """
                        docker run --rm -v /var/run/docker.sock:/var/run/docker.sock \\
                        aquasec/trivy image --exit-code 1 --severity CRITICAL \\
                        ${DOCKER_REGISTRY}/${IMAGE_NAME}:${BUILD_NUMBER}
                    """
                    
                    docker.withRegistry("https://${DOCKER_REGISTRY}", 'registry-credentials') {
                        image.push()
                        image.push('latest')
                    }
                }
            }
        }
        
        stage('IaC Scan') {
            when {
                changeset "terraform/**/*"
            }
            steps {
                sh '''
                    docker run --rm -v $(pwd):/tf bridgecrew/checkov \
                    --directory /tf/terraform \
                    --compact
                '''
            }
        }
        
        stage('Deploy to Staging') {
            steps {
                sh 'kubectl apply -f k8s/staging/'
                sh 'kubectl rollout status deployment/app -n staging'
            }
        }
        
        stage('DAST') {
            steps {
                sh '''
                    docker run --rm -v $(pwd):/zap/wrk/:rw \
                    -t owasp/zap2docker-stable zap-baseline.py \
                    -t https://staging.app.company.com \
                    -r dast-report.html
                '''
                publishHTML([allowMissing: false, alwaysLinkToLastBuild: true, 
                    keepAll: true, reportDir: '.', reportFiles: 'dast-report.html', 
                    reportName: 'DAST Report'])
            }
        }
    }
    
    post {
        always {
            // Clean workspace
            cleanWs()
            
            // Notify security team of critical findings
            script {
                if (currentBuild.result == 'FAILURE') {
                    emailext (
                        subject: "Security Pipeline Failed: ${env.JOB_NAME} - ${env.BUILD_NUMBER}",
                        body: "Critical security issues detected. Check console output.",
                        to: "${env.CHANGE_AUTHOR_EMAIL}, security-team@company.com"
                    )
                }
            }
        }
    }
}
```

---

## 15.2 Security as Code: Policy-as-Code and Infrastructure-as-Code

**Security as Code (SaC)** treats security policies as version-controlled, testable, and automated artifacts—just like application code. This eliminates manual configuration drift and ensures consistent enforcement across environments.

### Open Policy Agent (OPA)

OPA is a general-purpose policy engine that enables unified policy enforcement across the stack—from Kubernetes admission control to API authorization.

**Rego Policy Language Basics:**

```rego
# kubernetes-security.rego
package kubernetes.admission

import future.keywords.if
import future.keywords.in

# Deny containers running as root
deny[msg] if {
    input.request.kind.kind == "Pod"
    container := input.request.object.spec.containers[_]
    not container.securityContext.runAsNonRoot
    msg := sprintf("Container %s must set runAsNonRoot: true", [container.name])
}

# Deny privileged containers
deny[msg] if {
    input.request.kind.kind == "Pod"
    container := input.request.object.spec.containers[_]
    container.securityContext.privileged
    msg := sprintf("Container %s must not be privileged", [container.name])
}

# Require resource limits
deny[msg] if {
    input.request.kind.kind == "Pod"
    container := input.request.object.spec.containers[_]
    not container.resources.limits.memory
    msg := sprintf("Container %s must have memory limits defined", [container.name])
}

# Enforce image registry whitelist
allowed_registries := {"registry.company.com", "gcr.io/company-project"}

deny[msg] if {
    input.request.kind.kind == "Pod"
    container := input.request.object.spec.containers[_]
    image := container.image
    not starts_with_allowed_registry(image)
    msg := sprintf("Container %s uses image from unauthorized registry: %s", [container.name, image])
}

starts_with_allowed_registry(image) if {
    registry := allowed_registries[_]
    startswith(image, registry)
}
```

**Kubernetes Admission Controller Integration:**

```yaml
# OPA Gatekeeper deployment
apiVersion: config.gatekeeper.sh/v1alpha1
kind: Config
metadata:
  name: config
  namespace: gatekeeper-system
spec:
  sync:
    syncOnly:
      - group: ""
        version: "v1"
        kind: "Namespace"
      - group: ""
        version: "v1"
        kind: "Pod"
---
apiVersion: templates.gatekeeper.sh/v1beta1
kind: ConstraintTemplate
metadata:
  name: k8srequiredlabels
spec:
  crd:
    spec:
      names:
        kind: K8sRequiredLabels
      validation:
        openAPIV3Schema:
          properties:
            labels:
              type: array
              items: string
  targets:
    - target: admission.k8s.gatekeeper.sh
      rego: |
        package k8srequiredlabels
        violation[{"msg": msg}] {
          provided := {label | input.review.object.metadata.labels[label]}
          required := {label | label := input.parameters.labels[_]}
          missing := required - provided
          count(missing) > 0
          msg := sprintf("Missing required labels: %v", [missing])
        }
---
# Apply constraint
apiVersion: constraints.gatekeeper.sh/v1beta1
kind: K8sRequiredLabels
metadata:
  name: require-security-labels
spec:
  match:
    kinds:
      - apiGroups: [""]
        kinds: ["Pod"]
  parameters:
    labels: ["app.kubernetes.io/name", "security-level"]
```

### Terraform Policy as Code (Sentinel/OPA)

For cloud infrastructure, enforce security before resource creation:

```hcl
# sentinel.hcl - HashiCorp Sentinel configuration
policy "enforce-encryption" {
    enforcement_level = "hard-mandatory"
}

policy "restrict-instance-types" {
    enforcement_level = "soft-mandatory"
}
```

```python
# enforce-encryption.sentinel
import "tfplan"

# Check if S3 buckets have encryption enabled
main = rule {
    all tfplan.resources.aws_s3_bucket as _, buckets {
        all buckets as _, bucket {
            bucket.applied.server_side_encryption_configuration is not null
        }
    }
}

# Check EBS volumes are encrypted
ebs_encrypted = rule {
    all tfplan.resources.aws_ebs_volume as _, volumes {
        all volumes as _, volume {
            volume.applied.encrypted is true
        }
    }
}
```

**Using OPA with Terraform:**

```rego
# terraform.rego
package terraform.analysis

import input.tfplan as plan

# Deny public S3 buckets
deny[msg] {
    resource := plan.resource_changes[_]
    resource.type == "aws_s3_bucket"
    resource.change.after.acl == "public-read"
    msg := sprintf("S3 bucket %s must not be public", [resource.name])
}

# Require specific instance types
allowed_instance_types := {"t3.micro", "t3.small", "m5.large"}

deny[msg] {
    resource := plan.resource_changes[_]
    resource.type == "aws_instance"
    instance_type := resource.change.after.instance_type
    not allowed_instance_types[instance_type]
    msg := sprintf("Instance type %s not allowed", [instance_type])
}

# Enforce tags for cost tracking
deny[msg] {
    resource := plan.resource_changes[_]
    resource.type == "aws_instance"
    not resource.change.after.tags["Environment"]
    msg := "All instances must have Environment tag"
}
```

### CI/CD Policy Enforcement

Enforcing security gates in pipelines through code:

```yaml
# policy-check.yml
security_policies:
  - name: "No Critical CVEs"
    description: "Container images must not contain critical vulnerabilities"
    check: "trivy_scan"
    severity_threshold: "CRITICAL"
    action: "block"
  
  - name: "Signed Commits"
    description: "All commits must be GPG signed"
    check: "git_verify"
    action: "block"
  
  - name: "Branch Protection"
    description: "Main branch requires pull request reviews"
    check: "github_branch_protection"
    action: "warn"
  
  - name: "Secrets Scan Clean"
    description: "No hardcoded secrets in codebase"
    check: "gitleaks"
    action: "block"
```

---

## 15.3 Automated Security Testing in CI/CD

Modern DevSecOps pipelines integrate multiple testing methodologies, each detecting different vulnerability classes.

### SAST (Static Application Security Testing)

SAST analyzes source code without execution, identifying vulnerabilities like SQL injection, XSS, and buffer overflows.

**Semgrep (Lightweight, fast SAST):**

```yaml
# .semgrep.yml
rules:
  - id: sql-injection
    languages: [python]
    message: "Potential SQL injection detected"
    pattern-either:
      - pattern: |
          query = "..." + $X
          ...
          cursor.execute(query)
      - pattern: |
          query = f"...{$X}..."
          ...
          cursor.execute(query)
    severity: ERROR
  
  - id: hardcoded-credentials
    languages: [python, javascript, go]
    pattern-regex: (?i)(password|passwd|pwd)\s*=\s*['"][^'"]+['"]
    message: "Hardcoded credential detected"
    severity: WARNING
  
  - id: insecure-random
    languages: [python]
    pattern: random.random()
    fix: secrets.SystemRandom().random()
    message: "Insecure random number generator"
```

**Bandit (Python-specific):**

```bash
# Run bandit with specific tests
bandit -r ./src -f json -o bandit-report.json \
  --skip B101,B102 \
  --severity-level medium \
  --confidence-level medium
```

### DAST (Dynamic Application Security Testing)

DAST tests running applications externally, simulating attacker behavior.

**OWASP ZAP Automation:**

```python
# zap_automation.py
from zapv2 import ZAPv2
import time
import sys

class ZAPSecurityScanner:
    def __init__(self, api_key, target_url):
        self.zap = ZAPv2(apikey=api_key)
        self.target = target_url
        self.scan_policy = "Default Policy"
        
    def spider_scan(self):
        """Crawl the application"""
        scan_id = self.zap.spider.scan(self.target)
        while int(self.zap.spider.status(scan_id)) < 100:
            time.sleep(5)
        print(f"Spider completed. Found {len(self.zap.spider.results(scan_id))} URLs")
    
    def active_scan(self):
        """Attack discovered URLs"""
        scan_id = self.zap.ascan.scan(self.target, scanpolicyname=self.scan_policy)
        while int(self.zap.ascan.status(scan_id)) < 100:
            progress = self.zap.ascan.status(scan_id)
            print(f"Active scan progress: {progress}%")
            time.sleep(10)
    
    def check_risks(self):
        """Verify no high risks before deployment"""
        alerts = self.zap.core.alerts(baseurl=self.target)
        high_risks = [alert for alert in alerts if alert['risk'] == 'High']
        medium_risks = [alert for alert in alerts if alert['risk'] == 'Medium']
        
        print(f"High Risk Issues: {len(high_risks)}")
        print(f"Medium Risk Issues: {len(medium_risks)}")
        
        # Fail CI if high risks found
        if len(high_risks) > 0:
            for alert in high_risks:
                print(f"CRITICAL: {alert['name']} - {alert['description']}")
            sys.exit(1)
        
        return len(high_risks) == 0

# Usage in CI pipeline
if __name__ == "__main__":
    scanner = ZAPSecurityScanner(
        api_key="changeme",
        target_url="https://staging.example.com"
    )
    scanner.spider_scan()
    scanner.active_scan()
    scanner.check_risks()
```

### SCA (Software Composition Analysis)

SCA identifies vulnerabilities in open-source dependencies and checks license compliance.

**Dependency-Check Configuration:**

```xml
<!-- dependency-check-config.xml -->
<configuration>
    <failBuildOnCVSS>7</failBuildOnCVSS>
    <suppressionFiles>
        <suppressionFile>dependency-check-suppressions.xml</suppressionFile>
    </suppressionFiles>
    <analyzers>
        <assemblyAnalyzerEnabled>false</assemblyAnalyzerEnabled>
        <nugetconfAnalyzerEnabled>true</nugetconfAnalyzerEnabled>
    </analyzers>
    <retireJs>
        <filterNonVulnerable>true</filterNonVulnerable>
    </retireJs>
</configuration>
```

**Snyk Policy File:**

```yaml
# .snyk
version: v1.25.0
ignore:
  'SNYK-JS-LODASH-1018905':
    - '*':
        reason: 'No fix available, not exploitable in our context'
        expires: 2024-12-31T00:00:00.000Z
patch: {}
```

### IAST (Interactive Application Security Testing)

IAST instruments applications during testing, combining SAST and DAST advantages with minimal false positives.

**Contrast Security (Java Example):**

```xml
<!-- pom.xml -->
<plugin>
    <groupId>com.contrastsecurity</groupId>
    <artifactId>contrast-maven-plugin</artifactId>
    <version>2.13.0</version>
    <configuration>
        <username>${contrast.username}</username>
        <apiKey>${contrast.apiKey}</apiKey>
        <serviceKey>${contrast.serviceKey}</serviceKey>
        <orgUuid>${contrast.orgUuid}</orgUuid>
        <applicationName>SecureApp</applicationName>
        <serverName>CI-Server</serverName>
    </configuration>
</plugin>
```

```bash
# Run tests with IAST agent
mvn contrast:verify test
```

### Secrets Scanning

Prevent credential leakage through automated scanning:

**GitLeaks Configuration:**

```toml
# .gitleaks.toml
title = "GitLeaks Config"

[[rules]]
id = "AWS Access Key"
description = "AWS Access Key"
regex = '''(A3T[A-Z0-9]|AKIA|AGPA|AIDA|AROA|AIPA|ANPA|ANVA|ASIA)[A-Z0-9]{16}'''
tags = ["key", "AWS"]

[[rules]]
id = "Generic API Key"
description = "Generic API Key"
regex = '''[aA][pP][iI]_?[kK][eE][yY].*['|\"][0-9a-zA-Z]{32,45}['|\"]'''
secretGroup = 1
entropy = 3.7

[allowlist]
description = "Allowlisted files"
paths = [
    '''gitleaks.toml''',
    '''(.*?)(jpg|gif|doc|pdf|bin)$'''
]
```

---

## 15.4 Orchestration and Automation Tools for Security (SOAR)

**SOAR (Security Orchestration, Automation and Response)** platforms automate incident response workflows, threat intelligence management, and security tool integration. They reduce mean time to respond (MTTR) by eliminating manual coordination.

### SOAR Architecture Components

1. **Orchestration**: Connecting disparate security tools (SIEM, EDR, firewalls)
2. **Automation**: Playbooks that execute response actions without human intervention
3. **Response**: Case management, evidence collection, and remediation tracking

### Practical SOAR Implementation

Using open-source tools like Shuffle or Node-RED, or enterprise platforms like Splunk SOAR:

**Incident Response Playbook (Python-based SOAR):**

```python
# soar_playbook.py
from typing import Dict, List
import asyncio
import json

class IncidentResponseOrchestrator:
    def __init__(self):
        self.siem = SIEMConnector()
        self.edr = EDRConnector()
        self.firewall = FirewallConnector()
        self.ticketing = JiraConnector()
        self.slack = SlackNotifier()
    
    async def handle_phishing_alert(self, alert: Dict):
        """
        Automated phishing response workflow
        """
        incident_id = await self.create_incident(alert)
        
        # Enrichment phase
        await self.slack.notify(f"Investigating phishing alert: {incident_id}")
        
        # Gather threat intelligence
        sender_ip = alert['sender_ip']
        reputation = await self.check_ip_reputation(sender_ip)
        
        if reputation['malicious_score'] > 0.8:
            # Automated containment
            await self.firewall.block_ip(sender_ip)
            await self.edr.isolate_endpoint(alert['recipient_endpoint'])
            
            # Forensic collection
            await self.edr.collect_memory_dump(alert['recipient_endpoint'])
            await self.siem.export_logs(
                query=f"source_ip={sender_ip}",
                time_range="24h"
            )
            
            # Update incident
            await self.update_incident(incident_id, {
                "status": "Contained",
                "actions_taken": ["IP blocked", "Endpoint isolated"],
                "threat_intel": reputation
            })
            
            await self.slack.notify(
                f"🚨 Phishing incident {incident_id} contained automatically"
            )
        else:
            await self.ticketing.create_task(
                assignee="security-analyst",
                description=f"Review low-confidence phishing alert: {incident_id}"
            )
    
    async def handle_malware_detection(self, alert: Dict):
        """
        Malware outbreak response
        """
        hash_value = alert['file_hash']
        
        # Check if hash known malicious
        vt_results = await self.virustotal_lookup(hash_value)
        
        if vt_results['positives'] > 10:
            # Search for all instances across enterprise
            infected_hosts = await self.edr.hunt_ioc(hash_value)
            
            # Mass containment
            containment_tasks = [
                self.edr.isolate_endpoint(host) 
                for host in infected_hosts
            ]
            await asyncio.gather(*containment_tasks)
            
            # YARA rule deployment
            yara_rule = await self.generate_yara_rule(alert['file_sample'])
            await self.edr.deploy_yara_rule(yara_rule)
            
            # Notification
            await self.slack.notify(
                f"Malware {hash_value} contained on {len(infected_hosts)} hosts"
            )

class AutomatedVulnerabilityRemediation:
    """
    Auto-remediation for low-risk vulnerabilities
    """
    
    async def patch_container_cve(self, image_tag: str, cve_id: str):
        """
        Rebuild container with patched base image
        """
        # Check if auto-remediation approved for this CVE
        if not await self.is_approved_for_auto_remediation(cve_id):
            return False
        
        # Trigger rebuild pipeline
        await self.ci_cd.trigger_build(
            component=image_tag,
            base_image_update=True,
            security_patch=cve_id
        )
        
        # Verify fix in new image
        scan_results = await self.scanner.scan_image(f"{image_tag}-patched")
        if cve_id not in scan_results['vulnerabilities']:
            await self.deploy_to_staging(image_tag)
            return True
        
        return False
    
    async def rotate_exposed_credentials(self, leak_alert: Dict):
        """
        Automated credential rotation on secret leak detection
        """
        secret_id = leak_alert['secret_id']
        
        # 1. Revoke immediately
        await self.vault.revoke_secret(secret_id)
        
        # 2. Generate new credential
        new_secret = await self.vault.generate_dynamic_credential(
            service=leak_alert['service'],
            ttl="90d"
        )
        
        # 3. Update CI/CD secrets
        await self.ci_cd.update_secret(
            secret_name=leak_alert['secret_name'],
            new_value=new_secret
        )
        
        # 4. Restart affected services
        await self.kubernetes.rolling_restart(
            label_selector=f"uses-secret={leak_alert['secret_name']}"
        )
        
        # 5. Audit trail
        await self.audit.log_credential_rotation(
            secret_id=secret_id,
            reason="Exposed in commit",
            automated=True
        )
```

### Threat Intelligence Automation

```python
# threat_intel_automation.py
class ThreatIntelProcessor:
    def __init__(self):
        self.misp = MISPConnector()
        self.splunk = SplunkConnector()
        self.firewalls = [PaloAltoConnector(), CloudflareConnector()]
    
    async def ingest_ioc_feed(self, feed_url: str):
        """
        Automated IOC ingestion and blocking
        """
        iocs = await self.download_feed(feed_url)
        
        for ioc in iocs:
            # Deduplicate
            if await self.misp.check_exists(ioc['value']):
                continue
            
            # Enrich
            enrichment = await self.enrich_ioc(ioc)
            
            # Score
            risk_score = self.calculate_risk(enrichment)
            
            if risk_score > 70:
                # Auto-block on firewalls
                for fw in self.firewalls:
                    await fw.add_blocklist(ioc['value'], ioc['type'])
                
                # Hunt in logs
                hits = await self.splunk.hunt(ioc['value'])
                if hits > 0:
                    await self.create_incident(
                        title=f"Active IOC detected: {ioc['value']}",
                        severity="High",
                        hits=hits
                    )
            
            # Store in MISP
            await self.misp.create_event(ioc, enrichment, risk_score)
```

---

## 15.5 Case Studies: Building a DevSecOps Pipeline from Scratch

### Case Study 1: Cloud-Native Microservices Platform

**Scenario**: Building a secure pipeline for a Kubernetes-based microservices architecture handling financial data (PCI-DSS requirements).

**Architecture:**
```yaml
# complete-pipeline-example.yml
stages:
  - preflight
  - build
  - security-gates
  - deploy-staging
  - dast
  - deploy-production

preflight:
  script:
    - gitleaks detect --source .
    - trufflehog filesystem .
    - checkov --file Dockerfile --framework dockerfile

build:
  script:
    - docker build --target builder -t app:build .
    - docker run --rm app:build npm test
    - docker build -t $CI_REGISTRY_IMAGE:$CI_COMMIT_SHA .

security-gates:
  parallel:
    - job: sast
      script:
        - semgrep --config=auto --json --output=semgrep.json .
        - sonar-scanner -Dsonar.qualitygate.wait=true
    
    - job: sca
      script:
        - npm audit --audit-level moderate
        - snyk test --severity-threshold=high
    
    - job: container-scan
      script:
        - trivy image --exit-code 1 --severity HIGH,CRITICAL $CI_REGISTRY_IMAGE:$CI_COMMIT_SHA
        - trivy image --format cyclonedx --output sbom.json $CI_REGISTRY_IMAGE:$CI_COMMIT_SHA
    
    - job: iac-scan
      script:
        - checkov --directory terraform/ --framework terraform
        - terraform plan -out=tfplan
        - opa eval --data policy.rego --input tfplan "data.terraform.analysis.deny"

deploy-staging:
  script:
    - helm upgrade --install app ./helm-chart 
        --set image.tag=$CI_COMMIT_SHA
        --values values-staging.yaml
    - kubectl rollout status deployment/app -n staging

dast:
  script:
    - zap-full-scan.py -t https://staging.app.com -g gen.conf
    - nikto -h https://staging.app.com -o nikto.html

deploy-production:
  when: manual
  script:
    - cosign verify --key cosign.pub $CI_REGISTRY_IMAGE:$CI_COMMIT_SHA
    - helm upgrade --install app ./helm-chart 
        --set image.tag=$CI_COMMIT_SHA
        --values values-prod.yaml
    - kubectl apply -f network-policies/
```

### Case Study 2: AI/ML Model Deployment Pipeline

**Scenario**: Secure pipeline for deploying Chapter 14's agentic AI systems with model validation.

```python
# mlsec_pipeline.py
class MLSecurePipeline:
    """
    DevSecOps pipeline for AI/ML models
    """
    
    def __init__(self):
        self.model_scanner = ModelScanner()
        self.bias_detector = BiasDetector()
        self.adversarial_tester = AdversarialTester()
    
    def stage_1_model_validation(self, model_path: str):
        """
        Security checks specific to ML models
        """
        # Check for model serialization attacks
        if model_path.endswith('.pkl'):
            raise SecurityException("Pickle format not allowed. Use SafeTensors.")
        
        # Verify model signature
        if not self.verify_model_signature(model_path):
            raise SecurityException("Model signature invalid")
        
        # Scan for adversarial vulnerabilities
        robustness_score = self.adversarial_tester.test_robustness(model_path)
        if robustness_score < 0.8:
            raise SecurityException(f"Model fails robustness tests: {robustness_score}")
    
    def stage_2_data_provenance(self, training_data_manifest: str):
        """
        Verify training data integrity (from Chapter 14)
        """
        with open(training_data_manifest) as f:
            manifest = json.load(f)
        
        # Verify dataset hash matches expected
        current_hash = self.compute_dataset_hash(manifest['dataset_path'])
        if current_hash != manifest['expected_hash']:
            raise SecurityException("Training data tampering detected")
        
        # Check for poisoning via anomaly detection
        anomalies = self.detect_training_anomalies(manifest['dataset_path'])
        if anomalies > 0.01:  # 1% anomaly threshold
            raise SecurityException(f"Potential data poisoning: {anomalies*100}% anomalies")
    
    def stage_3_bias_testing(self, model_path: str, test_data: str):
        """
        Fairness and bias validation
        """
        bias_report = self.bias_detector.analyze(model_path, test_data)
        
        # Check demographic parity
        for group, metrics in bias_report['demographic_parity'].items():
            if metrics['disparate_impact'] < 0.8:  # 80% rule
                raise SecurityException(
                    f"Bias detected in group {group}: DI={metrics['disparate_impact']}"
                )
    
    def deploy_with_canary(self, model_path: str):
        """
        Canary deployment with adversarial monitoring
        """
        # Deploy to 5% traffic
        self.kubernetes.canary_deploy(model_path, traffic_percent=5)
        
        # Monitor for adversarial inputs
        time.sleep(300)  # 5 minutes observation
        attack_rate = self.monitor.adversarial_detection_rate()
        
        if attack_rate > 0.001:  # 0.1% threshold
            self.kubernetes.rollback()
            raise SecurityException(f"Adversarial attack rate too high: {attack_rate}")
        
        # Gradual rollout
        self.kubernetes.scale_canary(100)
```

### Implementation Checklist

**Phase 1: Foundation (Weeks 1-2)**
- [ ] Implement Git pre-commit hooks (secrets scanning)
- [ ] Configure SAST tools (Semgrep/SonarQube)
- [ ] Set up dependency scanning (Snyk/Dependabot)
- [ ] Enforce branch protection rules

**Phase 2: Integration (Weeks 3-4)**
- [ ] Container image scanning in CI
- [ ] SBOM generation for all artifacts
- [ ] Image signing with Cosign
- [ ] IaC scanning (Checkov)

**Phase 3: Advanced (Weeks 5-6)**
- [ ] DAST integration in staging
- [ ] OPA policies for Kubernetes
- [ ] Automated compliance checks (CIS benchmarks)
- [ ] SOAR playbooks for incident response

**Phase 4: Optimization (Ongoing)**
- [ ] False positive tuning
- [ ] Pipeline performance optimization
- [ ] Developer security training integration
- [ ] Metrics dashboard (MTTR, vulnerability density)

---

## Summary and Transition to Chapter 16

In this chapter, we operationalized security, transforming theoretical controls into automated, enforceable pipeline stages. You learned that **DevSecOps** is not a tool but a methodology—one that embeds security verification into every commit, build, and deployment.

Through **CI/CD integration**, you implemented security gates that fail fast: secrets scanning prevents credential leakage at commit time; SAST identifies code vulnerabilities before compilation; SCA manages dependency risks; container scanning ensures deployment artifacts are hardened; and DAST validates runtime security post-deployment. The combination of **Security as Code**—using OPA for Kubernetes admission control and Sentinel for Terraform policy enforcement—ensures that security constraints follow infrastructure wherever it scales.

We explored **SOAR** platforms that orchestrate incident response, automating the containment of phishing attacks, the isolation of malware-infected endpoints, and the rotation of exposed credentials—reducing response times from hours to seconds. The case studies demonstrated practical implementations: a PCI-DSS compliant microservices pipeline with cryptographic artifact verification, and a specialized MLOps pipeline extending Chapter 14's AI security controls into automated bias testing and adversarial robustness validation.

However, not all systems fit the cloud-native, CI/CD-deployed model we have focused on thus far. Critical infrastructure—power grids, manufacturing systems, medical devices—operates under constraints of legacy protocols, real-time requirements, and physical safety implications that demand specialized security architectures. These **Operational Technology (OT)** and **Industrial Control Systems (ICS)** environments face unique threats: air-gap jumping malware like Stuxnet, protocol-specific attacks against Modbus or OPC-UA, and the convergence of IT and OT networks.

In **Chapter 16: Specialized Security Domains**, we venture beyond enterprise software into **Industrial Control Systems (ICS/SCADA)** security, following the ISA/IEC 62443 standards for industrial automation. We will examine **Mobile Application Security** for iOS and Android platforms, addressing sandbox escapes and app store security. We explore **IoT Security** for resource-constrained devices, and finally **Blockchain and Web3 Security**, investigating smart contract vulnerabilities and consensus attacks. Each domain requires adapting the DevSecOps principles from this chapter to unique operational constraints—whether that means air-gapped deployments, battery-powered sensors, or immutable ledger transactions.